# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import randint


nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bonzoi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bonzoi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Define the path to the database file
db_path = "dataset/WB_disaster_Database.db"

# Create a SQLAlchemy engine to connect to the SQLite database
engine = create_engine(f'sqlite:///{db_path}')

# Retrieve data from the "WB_disaster_messages" table in the database
df = pd.read_sql_table("WB_disaster_messages", con=engine)

# Display the first few rows of the DataFrame
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df['message']
Y = df.iloc[:, 4:]

In [5]:
X.info()

<class 'pandas.core.series.Series'>
RangeIndex: 26216 entries, 0 to 26215
Series name: message
Non-Null Count  Dtype 
--------------  ----- 
26216 non-null  object
dtypes: object(1)
memory usage: 204.9+ KB


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """
    Tokenize input text.

    Parameters:
    text (str): Input text to be tokenized.

    Returns:
    list: A list of clean tokens.
    """
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Initialize WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Clean tokens
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
def build_model():
    """
    Builds a classifier model and tunes it using GridSearchCV.
    
    Returns:
    GridSearchCV: A classifier model tuned using GridSearchCV.
    """
    # Define the pipeline with vectorizer, transformer, and classifier
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),  # Tokenize and vectorize text
        ('tfidf', TfidfTransformer()),  # Apply TF-IDF transformation
        ('clf', MultiOutputClassifier(RandomForestClassifier()))  # Multi-output classifier with Random Forest
    ])
    
    # Define the parameters for grid search
    parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
        'clf__estimator__n_estimators': [10, 50],  # number of trees in the forest
        'clf__estimator__min_samples_split': [2, 5, 10]  # minimum number of samples required to split a node
    }
    
    # Create GridSearchCV
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
# Step 1: Define and call the build_model function to obtain the pipeline object
pipeline = build_model()

# Step 2: Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)  # Adjust test_size as needed

# Step 3: Train the pipeline on the training data
pipeline.fit(X_train, y_train)

c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [132]:
y_pred = pipeline.predict(X_test)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [79]:
def test_model(y_test, y_pred):
    
    """
    Function to iterate through columns and call sklearn classification report on each.
    """
    for index, column in enumerate(y_test):
        print(column, classification_report(y_test[column], y_pred[:, index]))

In [80]:
test_model(y_test, y_pred)

related               precision    recall  f1-score   support

           0       0.73      0.26      0.38      1536
           1       0.80      0.97      0.88      4969
           2       0.71      0.10      0.18        49

    accuracy                           0.80      6554
   macro avg       0.75      0.44      0.48      6554
weighted avg       0.78      0.80      0.76      6554

request               precision    recall  f1-score   support

           0       0.89      0.99      0.94      5417
           1       0.89      0.44      0.59      1137

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


money               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6390
           1       1.00      0.02      0.04       164

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.98      0.98      0.96      6554

missing_people               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6493
           1       1.00      0.02      0.03        61

    accuracy                           0.99      6554
   macro avg       1.00      0.51      0.51      6554
weighted avg       0.99      0.99      0.99      6554

refugees               precision    recall  f1-score   support

           0       0.97      1.00      0.98      6338
           1       0.50      0.02      0.04       216

    accuracy                           0.97      6554
   macro avg       0.73      0.51      0.51      6554
weighted avg       0.95      0.97      0.95 

c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\b

### 6. Improve your model
Use grid search to find better parameters. 

In [93]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000210AF979300>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000210AF979300>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [105]:
# specify parameters for grid search 
parameters = {
    'clf__estimator__warm_start' : [50, 100]
}

In [106]:
# create grid search object 
cv = GridSearchCV(pipeline, param_grid=parameters)

cv

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000210AF979300>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__warm_start': [50, 100]})

In [107]:
cv.fit(X_train, y_train)

c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\s

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000210AF979300>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__warm_start': [50, 100]})

In [108]:
cv.best_params_

{'clf__estimator__warm_start': 50}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [109]:
y_pred = cv.predict(X_test)

In [110]:
def test_model(y_true, y_pred):
    """
    Evaluate the model performance using various metrics.

    Parameters:
    y_true : numpy.ndarray of shape (n_samples, n_outputs)
        True labels.
    y_pred : numpy.ndarray of shape (n_samples, n_outputs)
        Predicted labels.
    """
    metrics = {}

    # Iterate over columns (outputs)
    for i in range(y_true.shape[1]):
        true_col = y_true[:, i]
        pred_col = y_pred[:, i]

        accuracy = accuracy_score(true_col, pred_col)
        precision = precision_score(true_col, pred_col, average='weighted')
        recall = recall_score(true_col, pred_col, average='weighted')
        f1 = f1_score(true_col, pred_col, average='weighted')

        # No column names available for numpy arrays
        column_name = f'Output_{i+1}'
        metrics[column_name] = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        }

        print(f"Metrics for {column_name}:")
        print("Accuracy: {:.2f}".format(accuracy))
        print("Precision: {:.2f}".format(precision))
        print("Recall: {:.2f}".format(recall))
        print("F1 Score: {:.2f}".format(f1))

    return metrics

# Now you can call test_model with y_test and y_pred
metrics = test_model(y_test.values, y_pred)



Metrics for Output_1:
Accuracy: 0.80
Precision: 0.79
Recall: 0.80
F1 Score: 0.76
Metrics for Output_2:
Accuracy: 0.89
Precision: 0.89
Recall: 0.89
F1 Score: 0.88
Metrics for Output_3:
Accuracy: 1.00
Precision: 0.99
Recall: 1.00
F1 Score: 0.99
Metrics for Output_4:
Accuracy: 0.77
Precision: 0.77
Recall: 0.77
F1 Score: 0.76
Metrics for Output_5:
Accuracy: 0.92
Precision: 0.90
Recall: 0.92
F1 Score: 0.89
Metrics for Output_6:
Accuracy: 0.95
Precision: 0.95
Recall: 0.95
F1 Score: 0.93
Metrics for Output_7:
Accuracy: 0.97
Precision: 0.97
Recall: 0.97
F1 Score: 0.96
Metrics for Output_8:
Accuracy: 0.98
Precision: 0.97
Recall: 0.98
F1 Score: 0.98
Metrics for Output_9:
Accuracy: 0.97
Precision: 0.96
Recall: 0.97
F1 Score: 0.96
Metrics for Output_10:
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1 Score: 1.00
Metrics for Output_11:
Accuracy: 0.94
Precision: 0.94
Recall: 0.94
F1 Score: 0.93


c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for Output_12:
Accuracy: 0.93
Precision: 0.92
Recall: 0.93
F1 Score: 0.92
Metrics for Output_13:
Accuracy: 0.93
Precision: 0.93
Recall: 0.93
F1 Score: 0.92
Metrics for Output_14:
Accuracy: 0.99
Precision: 0.98
Recall: 0.99
F1 Score: 0.98
Metrics for Output_15:
Accuracy: 0.98
Precision: 0.98
Recall: 0.98
F1 Score: 0.96
Metrics for Output_16:
Accuracy: 0.99
Precision: 0.99
Recall: 0.99
F1 Score: 0.99
Metrics for Output_17:
Accuracy: 0.97
Precision: 0.96
Recall: 0.97
F1 Score: 0.95
Metrics for Output_18:
Accuracy: 0.96
Precision: 0.95
Recall: 0.96
F1 Score: 0.94
Metrics for Output_19:
Accuracy: 0.86
Precision: 0.82
Recall: 0.86
F1 Score: 0.80
Metrics for Output_20:
Accuracy: 0.94
Precision: 0.88
Recall: 0.94
F1 Score: 0.90
Metrics for Output_21:
Accuracy: 0.96
Precision: 0.95
Recall: 0.96
F1 Score: 0.94
Metrics for Output_22:
Accuracy: 0.95
Precision: 0.94
Recall: 0.95
F1 Score: 0.93
Metrics for Output_23:
Accuracy: 0.98
Precision: 0.98
Recall: 0.98
F1 Score: 0.97
Metrics for Outp

c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bonzoi\AppData\Local\anaconda3\Lib\site-

Metrics for Output_34:
Accuracy: 0.98
Precision: 0.98
Recall: 0.98
F1 Score: 0.97
Metrics for Output_35:
Accuracy: 0.95
Precision: 0.93
Recall: 0.95
F1 Score: 0.92
Metrics for Output_36:
Accuracy: 0.87
Precision: 0.87
Recall: 0.87
F1 Score: 0.85


In [111]:
accuracy = (y_pred == y_test).mean()
accuracy

related                   0.796765
request                   0.892432
offer                     0.996186
aid_related               0.769149
medical_help              0.922643
medical_products          0.953311
search_and_rescue         0.973146
security                  0.984284
military                  0.970095
child_alone               1.000000
water                     0.944919
food                      0.927220
shelter                   0.932865
clothing                  0.987183
money                     0.975435
missing_people            0.990845
refugees                  0.967501
death                     0.958193
other_aid                 0.860391
infrastructure_related    0.935612
transport                 0.955294
buildings                 0.948734
electricity               0.981080
tools                     0.993134
hospitals                 0.989777
shops                     0.997711
aid_centers               0.988404
other_infrastructure      0.953921
weather_related     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [117]:
import pickle
filename = 'Models/classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.